In [15]:
# Import the required libraries and methods
import os
import ollama
import json
from youtube_transcript_api import YouTubeTranscriptApi
from dotenv import load_dotenv
#from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
#import openai
#import requests
#from typing import List, Literal
#from langchain_core.tools import tool
#from langchain_openai import ChatOpenAI

In [16]:
#Ensuring the current working directly has the environment file with API keys

import os

current_working_directory = os.getcwd()
print(f"Jupyter Notebook's current working directory: {current_working_directory}")

# Check for the .env file in this directory
env_file_path = os.path.join(current_working_directory, '.env')
if os.path.exists(env_file_path):
    print(f"'.env' file found at: {env_file_path}")
    # Also print its content to be absolutely sure
    with open(env_file_path, 'r') as f:
        print("\n--- .env file content ---")
        print(f.read())
        print("-------------------------\n")
else:
    print(f"ERROR: '.env' file NOT found at: {env_file_path}")
    print("Please ensure your .env file is in the current working directory or a parent directory.")

Jupyter Notebook's current working directory: /Users/hshekar/langgraph_test
'.env' file found at: /Users/hshekar/langgraph_test/.env

--- .env file content ---
WEATHER_API_KEY="5dba1f60bb7c49a09db130437251606"
TAVILY_API_KEY="Qoa3xtvNMIB9fYCE2ZemJ9GZiRVnJWKz"
TOGETHER_API_KEY="Ad08267a811875d5a525a7e2c39c3260ce347105eaa3c559f626aae9f23a6fb1"
api_key="219b45490dc00d37f87d78f11942bf48287964620e6850b1ab33f1923cd246c3"

-------------------------



In [17]:
import os
from dotenv import load_dotenv
load_dotenv('touch.env')

api_key = os.environ["api_key"]

In [18]:
def get_youtube_transcript(video_id):
    """Fetches transcript for a given YouTube video ID."""
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id)
        return " ".join([t['text'] for t in transcript])
    except Exception as e:
        # It's better to return an empty string or raise an error 
        print(f"Error retrieving transcript for video ID {video_id}: {e}")
        return "" # Return empty string on error, so subsequent functions can handle it

In [19]:
# --- 2. extract_keywords_and_themes function ---
def extract_keywords_and_themes(story_text_from_video):
    """Uses gpt-4o model to extract summary, transcript, emotions, and keywords from a story."""
    if not story_text_from_video or len(story_text_from_video.strip()) < 10: # Add a basic check for minimal text
        return {"summary": "N/A", "transcript": "N/A", "emotions": [], "keywords": [], "error": "Insufficient text for analysis."}

    prompt = f"""
    Analyze the following story transcript and extract:
    NO OTHER TEXT, EXPLANATION, OR PREAMBLE IS ALLOWED.
    YOUR ENTIRE RESPONSE MUST BE A SINGLE, VALID JSON OBJECT.
    1. The summary of the video
    2. The entire transcript of the video with appropriate punctuation marks. for example, make sure to have a full stop 
       and comma appropriately after each sentence in the story.
    3. Key phrases of animals, birds, any other nouns like name, place  (e.g., goat, wolf, bed, treasure chest).
    4. Extract 30 keywords of animals, birds, any other objects  (e.g., goat, wolf, bed, treasure chest).

Story Transcript:
{story_text_from_video}

Your output MUST be a valid JSON object. Do NOT include any other text or explanation.
The JSON object MUST have the following structure:
{{
    "summary": "...",
    "transcript": "...",
    "emotions": ["...", "..."],
    "keywords": ["...", "..."]
}}
"""
    llm_output_content = "" # Initialize to avoid UnboundLocalError in error case
    try:
        # Initialize ChatOpenAI with the provided base_url, api_key, and model
        chat_model = ChatOpenAI(base_url="https://aiplatform.dev51.cbf.dev.paypalinc.com/cosmosai/llm/v1",api_key=api_key,
            model="gpt-4o",
            temperature=0.0 # Often good for structured output like JSON
        )

        # Create a HumanMessage from the prompt
        messages = [HumanMessage(content=prompt)]

        # Invoke the chat model
        response = chat_model.invoke(messages)

        # Extract the content from the AI's response
        llm_output_content = response.content

        # Parse the JSON response
        parsed_json = json.loads(llm_output_content)
        return parsed_json
        
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM output as JSON in extract_keywords_and_themes: {e}")
        print(f"Raw LLM output causing error: '{llm_output_content}'")
        return {"error": "Could not parse LLM output as JSON", "raw_output": llm_output_content}
    except Exception as e:
        print(f"An unexpected error occurred in extract_keywords_and_themes: {e}")
        return {"error": f"Unexpected error: {e}"}

In [29]:
# --- 3. generate_summary_and_yoga function (Final Step) ---

def generate_summary_and_yoga(story_text_from_video, summary, transcript, emotions, keywords):
    """
    Summarizes the story and suggests yoga poses based on extracted transcript, emotions, and keywords.
    Args:
        story_text_from_video (str): The sample transcript of the YouTube video.
        summary (str): the summarized version of the transcript.
        transcript (str): The main transcript extracted.
        emotions (list or str): The key emotion(s) extracted.
        keywords (list or str): Important keywords extracted.
    Returns:
        str: A formatted string containing the summary and yoga pose recommendations.
    """
    # Ensure inputs are in a usable string format for the prompt
    summary_str = ",".join(summary) if isinstance(summary, list) else summary
    transcript_str = ", ".join(transcript) if isinstance(transcript, list) else transcript
    emotions_str = ", ".join(emotions) if isinstance(emotions, list) else emotions
    keywords_str = ", ".join(keywords) if isinstance(keywords, list) else keywords

    # Handle cases where inputs might be empty or "N/A"
    summary_display = f"Main summary(s): {summary_str}" if summary_str and summary_str != "N/A" else ""
    transcript_display = f"Main transcript(s): {transcript_str}" if transcript_str and transcript_str != "N/A" else ""
    emotions_display = f"Key Emotion(s): {emotions_str}" if emotions_str and emotions_str != "N/A" else ""
    keywords_display = f"Important Keywords: {keywords_str}" if keywords_str and keywords_str != "N/A" else ""

    extracted_info_block = "\n".join(filter(None, [summary_display,transcript_display, emotions_display, keywords_display]))
    if extracted_info_block:
        extracted_info_block = "--- Extracted Information:\n" + extracted_info_block
    else:
        extracted_info_block = "--- No specific extracted information available."

    prompt = f"""
Please perform two tasks based on the following story transcript and extracted information:

1.  **Summarize the story** concisely in 3-5 sentences, focusing on the main narrative or message.
2.  **Recommend 30 specific yoga poses according to the sequence in the transcript that align with the important keywords
extracted and not based on emotions** . 

---
Story Transcript:
{story_text_from_video}

{extracted_info_block}
---
Your output should be structured clearly with "Summary:" and "Recommended Yoga Poses:" sections.

Example Output Format:
Summary:
This is a concise summary of the story. It captures the main points and overall message.

Recommended Yoga Poses along with keywords: Please ensure there is a **line break after each complete pose entry**
- **seaside**: Child's Pose (Balasana): This pose is suitable for moments of calm and reflection, reminiscent of a peaceful seaside.
- **surfing**: Warrior II (Virabhadrasana II): This pose embodies strength and balance, perfect for the dynamic action of surfing.
- **mountain**: Mountain Pose (Tadasana): A grounding pose that symbolizes stability and strength, ideal for moments of steadfastness.
- **tranquility**: Savasana (Corpse Pose): Promotes deep relaxation and peace, aligning with feelings of serenity.
- **journey**: Sun Salutation (Surya Namaskar): A dynamic series of poses representing progression and new beginnings.
- **flight**: Eagle Pose (Garudasana): Requires balance and focus, suitable for themes of freedom or soaring.
- **discovery**: Tree Pose (Vrksasana): Cultivates balance and concentration, reflecting new insights and self-awareness.\n
- **challenge**: Plank Pose (Phalakasana): Builds core strength and resilience, perfect for overcoming obstacles.\n
- **connection**: Downward-Facing Dog (Adho Mukha Svanasana): Connects breath with movement and grounds the body, fostering a sense of unity.
- **joy**: Dancer's Pose (Natarajasana): A beautiful, balancing pose that expresses grace and happiness, embodying triumphant moments.
""" 
    
    try:
        # Initialize ChatOpenAI with the provided base_url, api_key, and model
        chat_model = ChatOpenAI(base_url="https://aiplatform.dev51.cbf.dev.paypalinc.com/cosmosai/llm/v1",api_key=api_key,
            model="gpt-4o",
            temperature=0.0 # Often good for structured output like JSON
        )

        # Create a HumanMessage from the prompt
        messages = [HumanMessage(content=prompt)]

        # Invoke the chat model
        response = chat_model.invoke(messages)
        return response
    except Exception as e:
        print(f"Error during summarization or pose suggestion: {e}")
        return f"Error processing request: {e}"

In [30]:
# --- Example Usage to Demonstrate the Full Flow ---
if __name__ == "__main__":
    # Choose a YouTube video ID to test
    # You can find video IDs in the URL after "v=" (e.g., LhYtcadR9nw is for a speech, good for themes)
    # o5t8P52D26Q (5-Minute Meditation for Stress Relief)
    video_id = "KHzJVnY2egA" ### Replace with your desired video ID

    print(f"--- Processing YouTube Video ID: {video_id} ---")

    # Step 1: Get YouTube Transcript
    print("\n[Step 1/3] Extracting YouTube transcript...")
    story_transcript = get_youtube_transcript(video_id)

    if not story_transcript:
        print("Failed to get transcript. Exiting.")
    else:
        print(f"Transcript extracted (first 100 chars): {story_transcript[:100]}...")

    # Step 2: Extract Keywords and Themes
    print("\n[Step 2/3] Extracting keywords and themes...")
    extracted_info = extract_keywords_and_themes(story_transcript)

    if extracted_info.get("error"):
        print(f"Error in extraction: {extracted_info['error']}")
    if 'raw_output' in extracted_info:
        print(f"Raw LLM output (for debug): {extracted_info['raw_output']}")
        print("Exiting.")
    else:
        print("\nExtracted Information:")
        print(f"  Summary: {extracted_info.get('summary')}")
        print(f"  transcript: {extracted_info.get('transcript')}")
        print(f"  Emotions: {extracted_info.get('emotions')}")
        print(f"  Keywords: {extracted_info.get('keywords')}")
                   
    # Step 3: Generate Summary and Yoga Pose Suggestions
    print("\n[Step 3/3] Generating summary and yoga pose suggestions...")
    final_output = generate_summary_and_yoga(
    story_transcript,
    extracted_info.get("summary", "N/A"), # Pass summary
    extracted_info.get("transcript", "N/A"), # Pass theme
    extracted_info.get("emotions", []),  # Pass emotions (as list)
    extracted_info.get("keywords", [])   # Pass keywords (as list)
    )

    print("\n--- Final Output ---")
    print(final_output)
    print("\n--- End of Processing ---")

--- Processing YouTube Video ID: KHzJVnY2egA ---

[Step 1/3] Extracting YouTube transcript...
Transcript extracted (first 100 chars): join kids hat family [Music] [Applause] [Music] what is wrong tofu i don't want to go to school toda...

[Step 2/3] Extracting keywords and themes...

Extracted Information:
  Summary: The story revolves around a poor farmer who discovers a magical pot that multiplies anything placed inside it. Using the pot, he transforms his family's fortune, but their secret is eventually revealed, leading the king to confiscate the pot. The king's greed and curiosity result in his downfall, and the farmer decides to leave the pot behind, valuing the riches he has already gained. The story teaches the importance of letting go and being content.
  transcript: Join Kids Hat Family. What is wrong, Tofu? I don't want to go to school today. Jim and Jerry took all the tokens I had collected for the charity and submitted it as their own. Now they're going to win the apprecia